In [81]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


**Loading The Data**

In [82]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
#print(train.head)
targets = train['target']
train = train.drop(columns=['id', 'target'])
#print(test.head)
ids = test['id']
test = test.drop(columns=['id'])


**Principal Component Analysis**

First I am going to use PCA to reduce colinearity in the data, there are 300 variables and the discussion board have noted a lot of colinearity so first I am going to start with a dimensionality of 50

In [83]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca.fit(train.values)
print(train.values.shape)
print(test.shape)
print(pca.explained_variance_ratio_)
X = pca.transform(train.values)
transformed_test = pca.transform(test.values)

(250, 300)
(19750, 300)
[0.01431672 0.01393798 0.01359807 0.01318352 0.01312196 0.01278289
 0.01258938 0.0123625  0.01225155 0.01205223 0.01182797 0.01170305
 0.01161672 0.01146128 0.01126607 0.0111215  0.01084649 0.01078387
 0.01053695 0.01040434 0.01020549 0.01017119 0.01010496 0.00993469
 0.00987094 0.00972216 0.00959875 0.00948667 0.00933842 0.00919666
 0.00918252 0.00901263 0.00896871 0.00881031 0.00871228 0.00866302
 0.00853263 0.00843835 0.00838467 0.00823717 0.00818802 0.00803781
 0.00794957 0.00789526 0.00781256 0.0077439  0.00762756 0.00760538
 0.00750732 0.00741127]


**Random Forests**
Fitting Random Forests model to the data using K-Fold Cross Validation

In [84]:
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier

kf = KFold(n_splits=10)

scores = [] 

for train_index, test_index in kf.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = targets[train_index], targets[test_index]
    clf = RandomForestClassifier(n_estimators=27, max_depth=4,random_state=0)
    clf.fit(x_train, y_train)
    score = clf.score(x_test, y_test)
    scores.append(score)
    print(score)
print("Average Score: ", sum(scores)/10)
clf.fit(X, targets)
predictions = clf.predict(transformed_test)

    


0.72
0.92
0.64
0.88
0.64
0.72
0.6
0.72
0.68
0.6
Average Score:  0.712


**Write Out Predictions**



In [85]:
output = pd.DataFrame()
output['id'] = ids
output['target'] = predictions
output.to_csv('output.csv',index=None)